https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import keras.utils as ku
import numpy as np

In [6]:
tokenizer = Tokenizer()

In [9]:
data = open('data.txt').read()

In [39]:
def dataset_preparation(data):
    
    # basic cleanup
    corpus = data.lower().split('\n')

    # tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # create input sequences using list of tokens
    input_sequences = []

    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]

        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # create predictors and label
    predictors, label = input_sequences[:, :-1], input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

In [40]:
def create_model(predictors, label, max_sequence_len, total_words):
    
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

    model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
    print(model.summary)
    return model

In [37]:
def generate_text(seed_text, next_words, max_sequence_len):

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    
    return seed_text

In [41]:
data = open('data.txt').read()

In [43]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)
print (generate_text("we naughty", 3, max_sequence_len))

Epoch 1/100
48/48 [==============================] - 3s 61ms/step - loss: 3.7620 - acc: 0.0000e+00
Epoch 2/100
48/48 [==============================] - 0s 2ms/step - loss: 3.7588 - acc: 0.0625
Epoch 3/100
48/48 [==============================] - 0s 2ms/step - loss: 3.7558 - acc: 0.0625


C:\Users\Sony\Anaconda3\lib\site-packages\keras\callbacks.py:494: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 4/100
48/48 [==============================] - 0s 2ms/step - loss: 3.7534 - acc: 0.0625
Epoch 5/100
48/48 [==============================] - 0s 2ms/step - loss: 3.7496 - acc: 0.0625
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7450 - acc: 0.0625
Epoch 7/100
48/48 [==============================] - 0s 2ms/step - loss: 3.7385 - acc: 0.0625
Epoch 8/100
48/48 [==============================] - 0s 2ms/step - loss: 3.7308 - acc: 0.0625
Epoch 9/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7179 - acc: 0.0625
Epoch 10/100
48/48 [==============================] - 0s 2ms/step - loss: 3.7014 - acc: 0.0625
Epoch 11/100
48/48 [==============================] - 0s 2ms/step - loss: 3.6768 - acc: 0.0625
Epoch 12/100
48/48 [==============================] - 0s 2ms/step - loss: 3.6590 - acc: 0.0625
Epoch 13/100
48/48 [==============================] - 0s 2ms/step - loss: 3.6331 - acc: 0.0625
Epoch 14/100
48/48 [==============================] - 0s

48/48 [==============================] - 0s 1ms/step - loss: 1.8993 - acc: 0.5417
Epoch 90/100
48/48 [==============================] - 0s 2ms/step - loss: 1.9187 - acc: 0.4792
Epoch 91/100
48/48 [==============================] - 0s 1ms/step - loss: 2.0666 - acc: 0.4167
Epoch 92/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9868 - acc: 0.4375
Epoch 93/100
48/48 [==============================] - 0s 2ms/step - loss: 1.9657 - acc: 0.4375
Epoch 94/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8749 - acc: 0.5208
Epoch 95/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9496 - acc: 0.4792
Epoch 96/100
48/48 [==============================] - 0s 2ms/step - loss: 1.8704 - acc: 0.4792
Epoch 97/100
48/48 [==============================] - 0s 2ms/step - loss: 1.8103 - acc: 0.5208
Epoch 98/100
48/48 [==============================] - 0s 2ms/step - loss: 1.8717 - acc: 0.5000
Epoch 99/100
48/48 [==============================] - 0s 2ms/st

In [45]:
generate_text("cat and", 3, max_sequence_len)

'cat and kittens kittens we'